# Testing Proportions II: K Proportions
Source: https://web.williams.edu/Mathematics/sjmiller/public_html/BrownClasses/162/Handouts/StatsTests04.pdf

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2

In [2]:
#create toy data
math = np.random.binomial(1, 0.58, 600)
coding  = np.random.binomial(1, 0.51, 800)
econ  = np.random.binomial(1, 0.55, 600)

In [3]:
#combine in dict
udemy_dict = {'math': math, 'coding': coding, 'econ': econ}
for key in udemy_dict:
    print("{} mean: {}".format(key, np.mean(udemy_dict[key])))

math mean: 0.545
coding mean: 0.49125
econ mean: 0.5316666666666666


In [4]:
#orient by index then transpose to avoid same length error
df = pd.DataFrame.from_dict(udemy_dict, orient='index').T
#shuffle for randomness
df = df.sample(frac=1.0).reset_index(drop=True)
df.head()

,math,coding,econ
0,1.0,0.0,0.0
1,1.0,1.0,1.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,NaN,1.0,NaN


In [5]:
df.mean()

math      0.545000
coding    0.491250
econ      0.531667
dtype: float64

In [6]:
counts = df.count()
counts

math      600
coding    800
econ      600
dtype: int64

In [7]:
success = df.sum()
success

math      327.0
coding    393.0
econ      319.0
dtype: float64

In [8]:
failure = counts - success
failure

math      273.0
coding    407.0
econ      281.0
dtype: float64

In [9]:
df_stats = pd.DataFrame({'success': success, 'failure':failure, 'counts': counts})
df_stats

,success,failure,counts
math,327.0,273.0,600
coding,393.0,407.0,800
econ,319.0,281.0,600


In [10]:
def test_k_prop(c_r_table, alpha=0.05, theta_null=None):
    #calculcate degrees of freedom
    deg_freedom = c_r_table.shape[0]
    
    #check if theta specified
    if theta_null:
        theta = theta_null
        
    #calculcated pooled estimate otherwise
    else:
        #reduce deg of freedom
        deg_freedom -= 1
        print("Using {} degrees of freedom".format(deg_freedom))
        theta_hat = c_r_table['success'].sum() / c_r_table['counts'].sum()
        theta = theta_hat
        print("calculated pooled theta: {}".format(theta))
        
    
    #calculcate cutoff
    chi_critical = chi2.isf(alpha, deg_freedom)
    print("Evaluating test at chi square {}".format(chi_critical))
    
    #calculated expected values:
    e_win = c_r_table['counts'] * theta
    e_lose = c_r_table['counts'] * (1-theta)
    
    #create copy of dataframe
    df_test = c_r_table.copy()
    df_test['e_win'] = e_win
    df_test['e_fail'] = e_lose
    
    df_test['chi_win'] = ((df_test['success'] - df_test['e_win'])**2  ) / df_test['e_win'] 
    
    df_test['chi_fail'] = ((df_test['failure'] - df_test['e_fail'])**2 ) / df_test['e_fail']
    
    chi_test = df_test['chi_win'].sum() + df_test['chi_fail'].sum()
    
    if chi_test > chi_critical:
        print("Reject null hypothesis with {} > {}".format(chi_test, chi_critical))
    else:
        print("Maintain null hypothesis with {} < {}".format(chi_test, chi_critical))
    
    return chi_test, df_test
    

In [11]:
chi_test,df_test=  test_k_prop(df_stats)
df_test

Using 2 degrees of freedom
calculated pooled theta: 0.5195
Evaluating test at chi square 5.991464547107983
Maintain null hypothesis with 4.476475385728355 < 5.991464547107983


,success,failure,counts,e_win,e_fail,chi_win,chi_fail
math,327.0,273.0,600,311.7,288.3,0.751011,0.811967
coding,393.0,407.0,800,415.6,384.4,1.228970,1.328720
econ,319.0,281.0,600,311.7,288.3,0.170966,0.184842


# Testing r × c Contingency Tables

In [12]:
#create toy data again
math_premiun = np.random.binomial(100, 0.49)
coding_premium  = np.random.binomial(150, 0.55)
econ_premium  = np.random.binomial(50, 0.52)
premium = [math_premiun, coding_premium, econ_premium]
premium

[58, 89, 23]

In [13]:
#make copy of previous df
df_r_c = df_stats[['success','failure']].copy()
#insert premium
df_r_c['premium'] = premium
#update counts
df_r_c['counts'] = df_r_c.sum(1)

df_r_c

,success,failure,premium,counts
math,327.0,273.0,58,658.0
coding,393.0,407.0,89,889.0
econ,319.0,281.0,23,623.0


In [14]:
def r_c_expectations(r_c_table, cat_cols = ['success', 'failure', 'premium']):
    #get total count of observations
    n_tot = df_r_c['counts'].sum()

    #create a dataframe with categorical columns
    df_cats = df_r_c[cat_cols].copy()
    
    #number of times we observe an outcome (fail, pass, premium, etc..)
    f_j = df_cats.sum(0)

    #number of times we observe population i (math, coding, econ)
    n_i = df_cats.sum(1)
    
    #create an empty matrix of zeroes
    e_ij = np.zeros(df_cats.shape)
    
    for i in range(len(n_i)):
        for j in range(len(f_j)):
            e_ij[i,j] = (n_i[i] * f_j[j])/n_tot

    return e_ij

In [15]:
r_c_expectations(df_r_c)

array([[315.0516129 , 291.4       ,  51.5483871 ],
       [425.65483871, 393.7       ,  69.64516129],
       [298.29354839, 275.9       ,  48.80645161]])

In [16]:
def r_c_hyp_test(r_c_table, cat_cols = ['success', 'failure', 'premium'], alpha=0.01):
    
    #get expecation matrix
    e_ij = r_c_expectations(r_c_table)
    
    #define categorical df
    df_cats = df_r_c[cat_cols].copy()
    
    #calculcate squared difference divided by expecation at corresponding location
    chi_df = ((e_ij - df_cats)**2)/e_ij
    
    #get chi value for that df
    chi_test = chi_df.sum().sum()
    
    #calculcate degree of freedom
    r, c = df_cats.shape
    deg_freedom = (r-1)*(c-1)
    
    #calculcate cutoff
    chi_critical = chi2.isf(alpha, deg_freedom)
    
    if chi_test > chi_critical:
        print("Reject null hypothesis with {} > {}".format(chi_test, chi_critical))
    else:
        print("Maintain null hypothesis with {} < {}".format(chi_test, chi_critical))
        
    return chi_test

In [17]:
r_c_hyp_test(df_r_c)

Reject null hypothesis with 25.932574905211517 > 13.276704135987625


25.932574905211517